In [21]:
!pip install transformers gradio sentencepiece


In [20]:
from transformers import MarianMTModel, MarianTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import gradio as gr

# Load Chinese->English model and tokenizer
tokenizer_zh_en = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
model_zh_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# Load MBART multilingual model and tokenizer for English->Polish
tokenizer_en_pl = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model_en_pl = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

def translate_zh_to_pl_two_step(text):
    # Step 1: Chinese -> English
    inputs_zh_en = tokenizer_zh_en(text, return_tensors="pt", padding=True)
    translated_en = model_zh_en.generate(
        **inputs_zh_en,
        max_length=40,
        num_beams=4,
        early_stopping=True
    )
    english_text = tokenizer_zh_en.decode(translated_en[0], skip_special_tokens=True)

    # Step 2: English -> Polish using MBART
    tokenizer_en_pl.src_lang = "en_XX"
    encoded_en = tokenizer_en_pl(english_text, return_tensors="pt", padding=True)
    generated_tokens = model_en_pl.generate(
        **encoded_en,
        forced_bos_token_id=tokenizer_en_pl.lang_code_to_id["pl_PL"],
        max_length=40,
        num_beams=4,
        early_stopping=True
    )
    polish_text = tokenizer_en_pl.decode(generated_tokens[0], skip_special_tokens=True)

    return polish_text

# Create Gradio interface
iface = gr.Interface(
    fn=translate_zh_to_pl_two_step,
    inputs=gr.Textbox(lines=3, placeholder="输入中文句子..."),
    outputs="text",
    title="Chinese to Polish Translator",
    description="Translates Chinese sentences into Polish via English using a two-step model."
)

iface.launch()


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://39be94e1d746d274cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
